특정 비트코인 transaction 정보를 1초마다 한 번씩 확인해서 크롤링<br>
새로운 데이터일 경우에만 추가

In [1]:
import urllib.request
import json
import time
import pymysql
import dbConfig
import time
from dbConfig import *

In [2]:
def crawlingData(coinType, into_database_list):
    into_database_dict = {}
    transactionData_p = []
    transactionData_b = []
    checkData_p = ''
    checkUnit_p = ''
    checkprice_p = ''
    checkType_p = ''
    checkData_b = ''
    checkUnit_b = ''
    checkprice_b = ''
    checkType_b = ''
    
    url_p = 'https://poloniex.com/public?command=returnTradeHistory&currencyPair=BTC_'+coinType+'&limit=1'
    urlTicker_poloniex = urllib.request.urlopen(url_p)
    readTicker_poloniex = urlTicker_poloniex.read()
    t_poloniex = json.loads(readTicker_poloniex)
    checkData_p = t_poloniex[0]['date']
    checkUnit_p = t_poloniex[0]['amount']
    checkprice_p = t_poloniex[0]['rate']
    
    url_b = 'https://api.bithumb.com/public/recent_transactions/'+coinType+'?count=1'
    urlTicker_bithumb = urllib.request.urlopen(url_b)
    readTicker_bithumb = urlTicker_bithumb.read()
    t_bithumb = json.loads(readTicker_bithumb)
    checkData_b = t_bithumb['data'][0]['transaction_date']
    checkUnit_b = t_bithumb['data'][0]['units_traded']
    checkprice_b = t_bithumb['data'][0]['price']
    
    while(True):
        time.sleep(1)

        url_p = 'https://poloniex.com/public?command=returnTradeHistory&currencyPair=BTC_'+coinType+'&limit=1'
        urlTicker_poloniex = urllib.request.urlopen(url_p)
        readTicker_poloniex = urlTicker_poloniex.read()
        t_poloniex = json.loads(readTicker_poloniex)
        transactionData_p.append(t_poloniex)
        if (transactionData_p[len(transactionData_p)-1][0]['date'] != checkData_p
                or transactionData_p[len(transactionData_p)-1][0]['amount'] != checkUnit_p  
                or transactionData_p[len(transactionData_p)-1][0]['rate'] != checkprice_p
                or transactionData_p[len(transactionData_p)-1][0]['type'] != checkType_p):
            print("Poloneix--", t_poloniex, end="\n\n")
            checkData_p = t_poloniex[0]['date']
            checkUnit_p = t_poloniex[0]['amount']
            checkprice_p = t_poloniex[0]['rate']
            checkType_p = t_poloniex[0]['type']
            into_database_dict = {}
            into_database_dict['globalTradeID'] = t_poloniex[0]['globalTradeID']
            into_database_dict['date'] = t_poloniex[0]['date']
            if t_poloniex[0]['type'] == 'sell':
                into_database_dict['type'] = 'ask'
            elif t_poloniex[0]['date'] == 'buy':
                into_database_dict['type'] = 'bid'
            #into_database_dict['type'] = t_poloniex[0]['type']
            into_database_dict['rate'] = float(t_poloniex[0]['rate'])
            into_database_dict['amount'] = float(t_poloniex[0]['amount'])
            into_database_dict['total'] = float(t_poloniex[0]['total'])
            into_database_dict['exchange'] = 'pol'
            into_database_list.append(into_database_dict)
            
            
        url_b = 'https://api.bithumb.com/public/recent_transactions/'+coinType+'?count=1'
        urlTicker_bithumb = urllib.request.urlopen(url_b)
        readTicker_bithumb = urlTicker_bithumb.read()
        t_bithumb = json.loads(readTicker_bithumb)
        transactionData_b.append(t_bithumb)
        if (transactionData_b[len(transactionData_b)-1]['data'][0]['transaction_date'] != checkData_b
                or transactionData_b[len(transactionData_b)-1]['data'][0]['units_traded'] != checkUnit_b
                or transactionData_b[len(transactionData_b)-1]['data'][0]['price'] != checkprice_b
                or transactionData_b[len(transactionData_b)-1]['data'][0]['type'] != checkType_b):
            print("Bithumb--", t_bithumb, end="\n\n")
            checkData_b = t_bithumb['data'][0]['transaction_date']
            checkUnit_b = t_bithumb['data'][0]['units_traded']
            checkprice_b = t_bithumb['data'][0]['price']
            checkType_b = t_bithumb['data'][0]['type']
            into_database_dict = {}
            into_database_dict['globalTradeID'] = int(time.time()) # None
            into_database_dict['date'] = t_bithumb['data'][0]['transaction_date']
            into_database_dict['type'] = t_bithumb['data'][0]['type']
            into_database_dict['rate'] = float(t_bithumb['data'][0]['price'])
            into_database_dict['amount'] = float(t_bithumb['data'][0]['units_traded'])
            into_database_dict['total'] = float(t_bithumb['data'][0]['total'])
            into_database_dict['exchange'] ='bit'
            into_database_list.append(into_database_dict)
            

In [4]:
coinType = ['ETH', 'DASH', 'LTC', 'ETC', 'XRP', 'BCH', 'XMR', 'ZEC']
into_database_list = []
crawlingData(coinType[3], into_database_list)

KeyboardInterrupt: 